# Document Loaders & Text Splitters

- Document Loaders read raw sources (files, PDFs, web pages, DB rows, strings) into a uniform Document shape (page_content + metadata).
- Text Splitters chunk long text into smaller, overlapping pieces that fit model context windows and improve retrieval quality.

What we'll cover:
- Creating Documents (from strings and files)
- Popular splitters (RecursiveCharacterTextSplitter, TokenTextSplitter, MarkdownHeaderTextSplitter)
- Preserving metadata (page numbers, headers, source)
- Choosing chunk size & overlap sensibly

## Bootstrap

⚓--- Before proceeding futher it is very important you do the following: --- 👾

Select the 🗝 (key) icon in the left pane and include your OpenAI Api key with Name as "OPENAPI_KEY" and value as the key, and grant it notebook access in order to be able to run this notebook.

Run the below cells in the order they are in, before running further cells. Wait till a number appears in place of '*' or '[ ]'. Below the cell you should see "✅ Ready: Text Splitters and Document tutorial"

In [ ]:
!git clone https://github.com/vishvak1/langchain-colab-notebooks.git

In [ ]:
import os
os.chdir("langchain-colab-notebooks")

In [ ]:
!pip install -q langchain langchain-openai langchain-community pydantic pypdf

In [ ]:
# No API keys required for this notebook.
from langchain_core.documents import Document

# Loaders (community package)
from langchain_community.document_loaders import TextLoader, PyPDFLoader
# (Optional) PDF example — uncomment if you have local PDFs
# from langchain_community.document_loaders import PyPDFLoader

# Text splitters
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    TokenTextSplitter,
    MarkdownHeaderTextSplitter,
)

print("✅ Ready: Text Splitters and Document tutorial")

## What a Document looks like

A document wraps text and metadata.

In [ ]:
# Create Documents by hand (great for demos/tests)
docs = [
    Document(
        page_content="LangChain helps you compose LLM applications.",
        metadata={"source": "notes", "topic": "langchain", "order": 1}
    ),
    Document(
        page_content="LCEL uses the | operator to connect runnables into chains.",
        metadata={"source": "notes", "topic": "lcel", "order": 2}
    ),
]

docs

1. Each document has page_content (string) and metadata (dict).
2. Keep metadata meaningul (eg. section, page, url) as it will help in the data storage part later.

## Loading from files (Plain text or PDFs)

Here's an example of a simple text file loader

In [ ]:
# Save a sample file once (for demo)
sample_path = "data/sample_notes.txt"
with open(sample_path, "w", encoding="utf-8") as f:
    f.write("Line 1: LCEL composes steps with pipes.\n")
    f.write("Line 2: Text splitters break large text into chunks with overlap.\n")

# Load the text file into Documents
loader = TextLoader(sample_path, encoding="utf-8")
file_docs = loader.load()

file_docs[:2]

Using PyPDFLoader you can load PDFs.

In [ ]:
# Uncomment if you have a local PDF path
pdf_loader = PyPDFLoader("data/my_file.pdf")
pdf_docs = pdf_loader.load()  # one Document per page with metadata["page"]
pdf_docs[0], pdf_docs[0].metadata

- `TextLoader` -> for loading simple text files
- `PyPDFLoader` -> for loading documents in the form of Pdfs

## The workhorse aka RecursiveCharacterSplitter

The `RecursiveCharacterTextSplitter` splits a large text into smaller chunks by trying a list of separators such as `\n`, `\n\n`, ` ` in a specific order. It's designed to keep semantically related pieces of text together as much as possible.

The goal is to create text chunks smaller than a specified `chunk_size`. This is helpful when you have a document you'd like to split into chunks and create vector embeddings for those chunks to create a RAG system.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,        # target chars per chunk
    chunk_overlap=40,      # overlap helps keep context continuity
    separators=["\n\n", "\n", " ", ""],  # try big → small units
    length_function=len    # length in characters (default)
)

split_docs = text_splitter.split_documents(pdf_docs) # You can switch this file_docs if pdf_docs doesn't work for you
# Comment the above lien and uncomment the below line if you have a pdf loaded as mentioned in the previous cell
# split_docs = text_splitter.split_documents(pdf_docs)
len(split_docs), split_docs[:2]

### How it works?

1. Takes the wholes text
2. It tries to split the text using the first separator in its list (by default, this is a double newline: "\n\n"). This attempts to split the text into paragraphs.
3. After splitting, it looks at the resulting list of text chunks.
4. If a created chunk is smaller than the desired `chunk_size` it is added to the output.
5. If the chunk is bigger than the `chunk_size`, recursively tries the next splitter `/n` on that chunk and so on.

## Token aware splitting - TokenTextSplitter

When working with models with strict token limits, split by tokens instead of characters.

In [ ]:
tok_splitter = TokenTextSplitter(
    chunk_size=120,     # roughly 120 tokens
    chunk_overlap=20,   # in tokens
    encoding_name="cl100k_base"  # good default for many OpenAI models
)

token_docs = tok_splitter.split_documents(pdf_docs) # You can switch this to file_docs if pdf_docs doesn't work for you.
len(token_docs), token_docs[:2]

### How it works

1. Uses tokenizer estimates (via tiktoken) to count tokens.
2. Keep chunks well under your model’s max context to leave room for prompt + response.

## MarkdownHeaderTextSplitter

Markdown docs often have section headers. This splitter preserves headers in metadata so you can filter or display section-aware results.

This is usually paired with other splitters to limit `chunk_size` or token limits

In [ ]:
markdown_text = """# Project Alpha

Intro paragraph about Alpha.

## Features
- Fast startup
- Modular design

## Architecture
The system consists of a frontend, backend, and vector store.

### Backend
Uses Python and FastAPI.

### Vector Store
FAISS in-memory for development; switchable later.
"""

# First: split into sections by headers and carry header paths in metadata
md_header_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "h1"),
        ("##", "h2"),
        ("###", "h3"),
    ]
)
md_sections = md_header_splitter.split_text(markdown_text)

# You now have Documents with metadata like {"h1": "...", "h2": "..."}
md_sections[:3], md_sections[0].metadata

# Second: optionally run a character/token splitter to limit chunk size
md_chunker = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=40)
md_chunks = md_chunker.split_documents(md_sections)

len(md_chunks), md_chunks[:2]